# Packages

In [1]:
#-------------importing libraries
import numpy as np
import pandas as pd
from tensorflow.keras.models import load_model

2022-09-24 14:44:05.522833: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/openmpi3/gcc/3.1.3/lib:/cm/shared/apps/nccl2-cuda10.2-gcc8/2.8.4/lib:/cm/shared/apps/cudnn7.6-cuda10.2/7.6.5.32/lib64:/cm/local/apps/cuda/libs/current/lib64:/cm/shared/apps/cuda10.2/toolkit/10.2.89/targets/x86_64-linux/lib:/cm/local/apps/python37/lib:/cm/shared/apps/gcc8/8.4.0/lib:/cm/shared/apps/gcc8/8.4.0/lib32:/cm/shared/apps/gcc8/8.4.0/lib64:/cm/shared/apps/hdf5_18/1.8.18-mpi/lib:/cm/shared/apps/openblas/0.2.20/lib:/cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2022-09-24 14:44:05.522884: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


## Loading test data

In [2]:
X_test = pd.read_csv("Data/X_test.csv")
y_test = pd.read_csv("Data/y_test.csv")

# Loading Models
## Monte Carlo Dropout

### `.h5` format

In [3]:
model_mcd_h5 = load_model("Saved_Models/Monte_Carlo_Dropout/model_mcd.h5")
model_mcd_h5.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 24        
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 dense_1 (Dense)             (None, 16)                144       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 457
Trainable params: 457
Non-trainable pa

2022-09-24 14:44:08.482625: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /cm/shared/apps/openmpi3/gcc/3.1.3/lib:/cm/shared/apps/nccl2-cuda10.2-gcc8/2.8.4/lib:/cm/shared/apps/cudnn7.6-cuda10.2/7.6.5.32/lib64:/cm/local/apps/cuda/libs/current/lib64:/cm/shared/apps/cuda10.2/toolkit/10.2.89/targets/x86_64-linux/lib:/cm/local/apps/python37/lib:/cm/shared/apps/gcc8/8.4.0/lib:/cm/shared/apps/gcc8/8.4.0/lib32:/cm/shared/apps/gcc8/8.4.0/lib64:/cm/shared/apps/hdf5_18/1.8.18-mpi/lib:/cm/shared/apps/openblas/0.2.20/lib:/cm/shared/apps/slurm/current/lib64/slurm:/cm/shared/apps/slurm/current/lib64
2022-09-24 14:44:08.482660: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-09-24 14:44:08.482685: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to 

### Making predictions (MCD)

In [4]:
#-------------Distribution of predictions
def predict_dist(X, saved_model, num_samples):
    '''
    #----------Arguments/Inputs:
        X: The input 
        saved_model: The dropout model
        num_samples: Number of predictions to make
    
    #----------Output(s):
        Returns all model predictions as a numpy array
    '''
    preds = [saved_model(X, training=True) for _ in range(num_samples)]
    return np.hstack(preds)

#-------------Distribution and Mean of predictions
def predict_point(X, saved_model, num_samples):
    '''
    #----------Arguments/Inputs::
        X: The input 
        saved_model: The dropout model
        num_samples: Number of predictions to make
    #----------Output(s):
        Returns the mean and distribution of predictions
    '''
    pred_dist = predict_dist(X, saved_model, num_samples)
    return pred_dist.mean(axis=1), pred_dist
    #return stats.mode(pred_dist), pred_dist

In [5]:
n = 50 #number of predictions to make
y_pred_dist_h5 = predict_dist(X_test.values, model_mcd_h5, n)
y_pred_mcd_h5 = predict_point(X_test.values, model_mcd_h5, n)
y_pred_mcd_h5

(array([0.7321405 , 0.87435555, 0.73496336, ..., 0.9049857 , 0.7484761 ,
        0.7583228 ], dtype=float32),
 array([[0.73254466, 0.73206115, 0.7209013 , ..., 0.7308456 , 0.72595096,
         0.72985613],
        [0.84842473, 0.8800608 , 0.8685616 , ..., 0.86824155, 0.8762979 ,
         0.87477666],
        [0.7441313 , 0.74439955, 0.73934186, ..., 0.7367853 , 0.73708296,
         0.7288079 ],
        ...,
        [0.9119562 , 0.89893264, 0.9050331 , ..., 0.9215157 , 0.9146234 ,
         0.91581786],
        [0.76504004, 0.7588123 , 0.742355  , ..., 0.7402612 , 0.7469099 ,
         0.73611826],
        [0.7560649 , 0.7539736 , 0.75851893, ..., 0.77402675, 0.7721853 ,
         0.77319777]], dtype=float32))


### Protocol Buffer format
Saving a model in this format saves it into multiple files. In the code below `tf_SavedModel_mcd` is the folder containing those files.

In [6]:
model_mcd_pb = load_model("Saved_Models/Monte_Carlo_Dropout/tf_SavedModel_mcd")
model_mcd_pb.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 24        
                                                                 
 dropout (Dropout)           (None, 8)                 0         
                                                                 
 dense_1 (Dense)             (None, 16)                144       
                                                                 
 dropout_1 (Dropout)         (None, 16)                0         
                                                                 
 dense_2 (Dense)             (None, 16)                272       
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 457
Trainable params: 457
Non-trainable pa

### Making predictions (MCD)

In [7]:
y_pred_dist_pb = predict_dist(X_test.values, model_mcd_pb, n)
y_pred_mcd_pb = predict_point(X_test.values, model_mcd_pb, n)
y_pred_mcd_pb

(array([0.73386276, 0.8764646 , 0.73409647, ..., 0.9033225 , 0.75019133,
        0.75021094], dtype=float32),
 array([[0.7328186 , 0.7276838 , 0.7425178 , ..., 0.735227  , 0.72411156,
         0.7256087 ],
        [0.87960994, 0.8804644 , 0.88706166, ..., 0.8932644 , 0.8932431 ,
         0.8792416 ],
        [0.73229885, 0.72329426, 0.734357  , ..., 0.7250036 , 0.7313055 ,
         0.7633828 ],
        ...,
        [0.9051845 , 0.9148581 , 0.9074991 , ..., 0.86992383, 0.9087603 ,
         0.8961021 ],
        [0.78384745, 0.75780296, 0.7477795 , ..., 0.7439172 , 0.74085975,
         0.7411423 ],
        [0.74221694, 0.7402245 , 0.75584614, ..., 0.7353674 , 0.7651681 ,
         0.76236284]], dtype=float32))

**NOTE:** In the two cases above, we can simply do `model.predict()` to make single point predictions. This way, dropout is not utilized at prediction time.


## Deep Ensembles (DE)

DE creates several models, hence, saving the models using the `.h5` and `protocol buffer` formats store the models the same way. The two methods save all the individual models into separate `.h5` files in a folder. Thus, loading the models is the same for the two methods. We can check the model summary of any of the models loaded. We can also use any of the models to make single point predictions. Below, we load all models into a list and the use each of them to predict on the test set. We will then calculate the mean and standard deviation of predictions as the aggregate prediction and measure of uncertainty, respectively.

In [8]:
models_de = list()

import glob
#-------------Reading all files ending with .hdf5
models = sorted(glob.glob("Saved_Models/Deep_Ensembles/tf_SavedModel_de/*.hdf5"))

for file in models:
    models_de.append(load_model(file))

### Making predictions (DE)

In [9]:
%%time
predictions_de = pd.DataFrame()
for model in models_de:
    y_pred = pd.DataFrame(model.predict(X_test.values)[0].ravel())
    predictions_de = pd.concat([predictions_de, y_pred], axis=1)

CPU times: user 55min 29s, sys: 5min 38s, total: 1h 1min 8s
Wall time: 44min 32s


In [10]:
predictions_de.head()

0         0         0         0         0         0         0  \
0  0.731615  0.724239  0.732037  0.733419  0.713078  0.723757  0.723448   
1  0.877722  0.869864  0.873159  0.881924  0.871429  0.871090  0.867665   
2  0.722733  0.718189  0.720094  0.722330  0.719715  0.722514  0.719618   
3  0.876152  0.872106  0.880441  0.883364  0.878575  0.877845  0.871757   
4  0.737384  0.733163  0.736845  0.743910  0.746132  0.735859  0.733614   

          0         0         0  ...         0         0         0         0  \
0  0.728760  0.713061  0.734045  ...  0.711359  0.724094  0.712826  0.727574   
1  0.872247  0.872348  0.873546  ...  0.870423  0.871023  0.868172  0.869549   
2  0.718950  0.719201  0.720319  ...  0.717739  0.721207  0.719036  0.718042   
3  0.872004  0.878416  0.879061  ...  0.876075  0.868940  0.874702  0.871382   
4  0.736516  0.745797  0.739994  ...  0.744687  0.736958  0.744965  0.733093   

          0         0         0         0         0         0  
0  0.723969  0.727784  0.725792  0.725865  0.727391  0.724090  
1  0.868327  0.873905  0.874002  0.865587  0.869559  0.868421  
2  0.717301  0.725718  0.723644  0.712017  0.717343  0.717923  
3  0.868688  0.878040  0.876968  0.871035  0.873602  0.872371  
4  0.731865  0.738268  0.736418  0.731609  0.733635  0.732538  

[5 rows x 50 columns]

In [11]:
y_pred_mean_de = predictions_de.mean(axis = 1).ravel()
y_pred_mean_de

array([0.72367835, 0.8702468 , 0.7193959 , ..., 0.90677124, 0.75349283,
       0.7638224 ], dtype=float32)

In [12]:
y_pred_std_de = predictions_de.std(axis = 1).ravel()
y_pred_std_de

array([0.00599962, 0.00401134, 0.00316195, ..., 0.00306389, 0.00489022,
       0.00433429], dtype=float32)

**NOTE:** The predicted values have not been inverse transformed

# THE END